In [ ]:
#Task: ABSA
#Language: English
#Contact: Pranaydeep Singh <pranaydeep.singh@ugent.be>
#Last Update: 2024-05-13
#Requirements: flair, transformers, ipymarkup
#Encoding: UTF-8

# Aspect-Based Sentiment Analysis (ABSA) with FLAIR for CLS

This Notebook shows you how to perform ABSA, which consists of two sub-tasks.
- Task A: Extraction of **Aspects** from unlabelled text
- Task B: Finding the sentiment associated with each **Aspect**.

**Aspects** can be any entities of interest in the text for which we want to find out the associated sentiment. This mostly depends on the kind of data you are working on. For example, the **Aspects** in a news corpora might be person names and organizations, while **Aspects** in a dataset of restaurant reviews might be names of dishes and drinks.

For this reason, we recommend training your own AE (Task A: Aspect Extraction) system using this notebook as a guide. If you do not have any unlabelled data, you can also used the created model directly as shown in Section 2, however this will only work well if the data and therefore the **Aspects** are similar to the ones we have used here. The sample dataset used here mostly includes named entities, especially Flora and Fauna annotated as aspects.

❗ To understand and adapt this Notebook for your own use case, we expect a basic understanding of these concepts:

- The task of ABSA
- Fine-tuning
- Language Models
- Python and frequently used libraries (HuggingFace Transformers, FLAIR, scikit-learn, etc)



#### Installing the Dependencies

We will first install all the libraries needed to run this notebook, this might take a few minutes.

In [ ]:
!pip install flair transformers torch pandas matplotlib ipymarkup scikit-learn statistics tqdm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7436 sha256=4c0ad7ac0ca6048a1ffd2059fee57d41e697e9bb9693c7717bc0bf73180b7c64
  Stored in directory: /Users/artemis/Library/Caches/pip/wheels/26/3c/70/9467407f3aa90862061eadcd286627b23a8bab6789b667776f
Successfully built statistics
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installa

## Section 1: Fine-tuning a Model with your annotated dataset

In this section, we will cover how you can fine-tune models for the task of ABSA from scratch using a few tools like transformers, the FLAIR-NLP toolkit, etc.
If you do not have access to annotated data for ABSA and therefore want to directly use the trained model we have created, then head to Section 2.
As detailed above, ABSA consists of two tasks Aspect Extraction (Task A) and Sentiment Classification (Task B), let's look into Task A first.

### Task A: Aspect Extraction

The first and hardest task of ABSA, involves finding interesting aspects in unstructured text. The aspects can be a single word or a large phrase or named entity. This makes the task more difficult since there is no standard word length or limits for the aspects.  

#### A.1 Loading and checking your Data

Now we will load our annotated dataset and do some quick checks to see how it looks. We have fine-tuned on our sample dataset. To train on your data, replace the paths of the files with your personal created dataset. However, make sure it's in the same format or you  might run into issues.

In [ ]:
import pandas as pd

df = pd.read_csv('english_aspect_extraction.txt', sep='\t', header=None, names=['Token', 'Label'])
df.head()

,Token,Label
0,Author,O
1,Weber,B-ASPECT
2,",",I-ASPECT
3,Max,I-ASPECT
4,",",O


Let's also split our data into a training, validation and test parts while we are at it.

We first make a 80:20 split, and keep 80 percent of the data for training the model.

From the 20 percent we wil divide it in half, and use 10 percent for validation and 10 percent for testing.

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_test_df = train_test_split(df, test_size=0.2, shuffle=False)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, shuffle=False)

train_df.to_csv('train.txt', sep='\t', header=False, index=False)
val_df.to_csv('val.txt', sep='\t', header=False, index=False)
test_df.to_csv('test.txt', sep='\t', header=False, index=False)

As you can see, we have successfully loaded our data. The **token** column refers to the current token in question, the **Label** column refers to the label of the current token. There are 3 labels:

- **B-ASPECT**: A token that marks the beginning of an aspect
- **I-ASPECT**: A token that is inside an aspect
- **O**: A token that is not part of any aspect

This data format is called the IOB (Inside, Outside, Beginning) format, and this is how most aspect extraction datasets are stored.

If your data is in another format, please refer to the [notebook on data conversion](https://teams.microsoft.com/l/message/19:32c7ccd6-db3e-49e0-ba27-87ec6aa57819_5a8f8965-f480-4f16-bb21-4d0b35a6246b@unq.gbl.spaces/1715587586609?context=%7B%22contextType%22%3A%22chat%22%7D)

#### A.2 Sequence Tagger with Flair-NLP: Loading Modules, Data & Defining Parameters

[Flair-NLP](https://github.com/flairNLP/flair) is a widely used toolkit for training advanced sequence tagger models for various applications like Named Entity Recognition, Aspect Based Sentiment Analysis, etc. The library provides a variety of options to train your models like word embeddings, pre-trained transformers, conditional random fields (CRFs), etc. Please refer to the [documentation of FLAIR](https://flairnlp.github.io/docs) for a more detailed overview of it's capabilities.

In this particular example, we will use one of the options available in FLAIR to train our own Sequence Tagger using a pre-trained Transformer model.

Let's begin by loading the necessary modules of Flair.

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data import Sentence
import numpy as np

Now let's begin by loading out data into Flair

In [ ]:
columns = {0: 'text', 1: 'label'} # This specifies which column is the text and which column is the label. In our case, the text is in the first column and the label is in the second column.

corpus: Corpus = ColumnCorpus("./", columns,
                              train_file='train.txt',
                              dev_file='val.txt',
                              test_file='test.txt')

2024-05-13 15:56:01,544 Reading data from .
2024-05-13 15:56:01,545 Train: train.txt
2024-05-13 15:56:01,546 Dev: val.txt
2024-05-13 15:56:01,547 Test: test.txt


Now let's define a few variables for our training. These are some key parameters so some additional information about them is added in the comments. Be sure to read them so you can make appropriate choices for your setting.

In [ ]:
model_name = 'emanjavacas/MacBERTh' #This is the pre-trained model we are going to use. You can change this to any other model from the transformers library. To look for available models, you can visit https://huggingface.co/models
fine_tune = False #If you want to fine-tune the model, set this to True. If you want to use the model as it is, set this to False. Fine-tuning results in better performance but requires more computational resources and time.
hidden_size = 256 #This is the size of the hidden layer of the model. You can change this to any other value depending on the computational resources you have. If your training is taking too long, experiment with reducing this number.
use_crf = False #If you want to use a CRF layer on top of the model, set this to True. This is recommended for sequence labeling tasks like NER. If you set this to False, the model will use a softmax layer for classification.
output_model_path = 'en_aspect_extraction_model' #This is the path where the trained model will be saved.

#### A.3 Sequence Tagger with Flair-NLP: Final Setup & Training

Perfect, now let's setup all the parameters and details of our model to-be trained soon!

In [ ]:
# 1. which column do we want to predict?
label_type = 'label'

# 3. make the label dictionary from the corpus, i.e. a mapping of labels to numbers
label_dict = corpus.make_label_dictionary(label_type=label_type, )
print(label_dict)

# 4. initialize fine-tuneable transformer embeddings WITH document context. These are the embeddings that will be used for classifying the tokens into aspects.
embeddings = TransformerWordEmbeddings(model = model_name,
                                       layers="-1", #ONLY USE THE LAST LAYER (common practice, but can experiment with other layers)
                                       subtoken_pooling="first",
                                       fine_tune= fine_tune,
                                       use_context=True, #document context is considered during the embedding process (surrounding words, ...)
                                       )

# 5. initialize our sequence tagger, you can experiment with the hyperparameters here to suit your needs. Some tinkering might help you reach better performance for your dataset.
tagger = SequenceTagger(hidden_size=hidden_size,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='bio',
                        use_crf=use_crf,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)



2024-05-13 16:10:01,514 Computing label dictionary. Progress:


1it [00:00,  1.86it/s]

2024-05-13 16:10:02,055 Dictionary created for label 'label' with 3 values: ASPECT (seen 227097 times), nan (seen 22709 times)


Dictionary with 3 tags: <unk>, ASPECT, nan
2024-05-13 16:10:04,505 SequenceTagger predicts: Dictionary with 9 tags: O, S-ASPECT, B-ASPECT, E-ASPECT, I-ASPECT, S-nan, B-nan, E-nan, I-nan


Great, everything seems to be ready, it's time to start training. Remember, this can take quite a while, so grab yourself a cup of coffee in the meantime!

In [ ]:
trainer.fine_tune(output_model_path,
                  learning_rate=5.0e-6, # Can be tuned for your data
                  mini_batch_size=4, # Should be adjusted based on your computation resources. If the code crashes due to memory errors, reduce it. If you have a big GPU, you can increase it to speed up training.
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

#### A.4 Testing the Model


Our model is ready, want to test it on a random sentence? Let's do it!

In [ ]:
sentence = Sentence('I saw a weeping willow while passing down the alleyway. The thorns of the roses reminded me of home, in Alabama.')

# predict aspect tags
tagger.predict(sentence)
print(sentence)

Sentence: "I saw a weeping willow while passing down the alleyway . The thorns of the roses reminded me of home , in Alabama ." → ["I"/ASPECT, "saw"/nan, "a"/nan, "weeping"/nan, "willow"/nan, "while"/nan, "passing"/nan, "down"/nan, "the"/nan, "alleyway"/nan, "."/nan, "The"/nan, "thorns"/nan, "of"/nan, "the"/nan, "roses"/nan, "reminded"/nan, "me"/nan, "of"/nan, "home"/nan, ","/nan, "in"/ASPECT, "Alabama"/ASPECT, "."/ASPECT]


As, you can see, even with some conservative settings we are able to have a pretty good aspect extraction model!

To make this model even better, you can try setting **fine_tune** to **True**, increasing the **hidden_size** and setting **use_crf** to True.

You can also further tune the hyper-parameters such as **learning_rate** since the optimal values can vary significantly depending on the model and data.

If you want to use this model to extract aspects from a large file, you can check out Section 2.

### Task B: Sentiment Classification

Since our model for Task A is capable of identifying interesting aspects in a sentence, we can move on to training a system for the next task of ABSA, ie. Sentiment Classification.
In this task we want to associate a sentiment label (eg: Positive, Negative, Neutral) to each aspect found by our model from Task A.

#### B.1 Loading and checking your Data

Once more, we will begin by first loading our data. The format of this file will look different than the previous file. This time we also need sentiment information for the aspects.
Our sample dataset for stored in the Comma-seperated Values (CSV) format. Your data might look different so make sure to adapt this code for your data format.

In [ ]:
import pandas as pd

df = pd.read_csv('data/English_asp_null_sent.csv')

df.rename(columns = {"text": "aspect", "_sentence_text": "sentence_1", "annotation": "category"}, inplace = True) # We will rename the columns to make them more readable
df.drop(['Unnamed: 0', 'source_file', 'sentence', 'begin', 'end', 'annotator',     # We will drop the columns that are not needed
       '_annotation', 'aspect_cat'], axis=1, inplace=True)

df.dropna(inplace=True) # Remove rows with missing values
df.head()

,aspect,sentence_1,category,sentiment_cat
0,breeze,"In the P.M. had a moderate breeze at East , wh...",ASPECT,3.0
1,Cloudy weather,At Midnight the wind came to South-South-West ...,ASPECT,4.0
2,Gale,Cloudy weather ; Winds at South-West and South...,ASPECT,4.0
3,Gale,Had a steady brisk Gale at South-South-West wi...,ASPECT,4.0
4,Gales,"Fresh Gales at South , which in the A.M. veer ...",ASPECT,4.0


As you can see, the data format looks quite different this time. We have the full text in the **sentence_1** column. The aspect is stored in the **aspect** column. While the sentiment label we will use is in the **sentiment_cat** column. The sentiment labels follow the standard range from 1 to 5, 5 being extremely positive and 1 being extremely negative.

Lastly, now we will convert the data to a simpler format, ie. lists for easier processing in the future.

In [ ]:
sentence_list = list(df["sentence_1"])
aspect_list = list(df["aspect"])
tag_list = list(df["sentiment_cat"])

raw_data = [[sent, asp, tag] for sent, asp, tag in zip(sentence_list, aspect_list, tag_list)]

#### B.2 Load model & create tagset

First let's define a few variables and load the model we will use.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

model_name = 'emanjavacas/MacBERTh' #This is the pre-trained model we are going to use. You can change this to any other model from the transformers library. To look for available models, you can visit https://huggingface.co/models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # This will automatically use the GPU if it is available, otherwise it will use the CPU.

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

We need to assign a number to each sentiment label so the model can interpret it. In Task A this was done using the **corpus.make_label_dictionary** functionality, but that only works when using flair. For this Task, we will be using **transformers**, so we will have to find another way.

In [ ]:
tags = list(set(df["sentiment_cat"]))
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}
print(idx2tag)

{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0}


#### B.3 Helper functions for manipulating data

Now we need to change the format of the data a bit. This is necessary because transformers break down words into smaller units called sub-words. This might cause some confusion, since an aspect like "John Oliver" annotated as sentiment **4.0**, might be broken down into 4 parts for example, "Jo", "hn", "Oli", "ver". In that case we also need to adjust our data so we can assign the sentiment label **4.0** to all those 4 parts. To know more about how tokenization in transformers works and the technical details around it please take a look at [this guide](https://docs.mistral.ai/guides/tokenization/).

In [ ]:
#This functions finds the index position of the aspect term in the sentence

def get_pos(sent_list,aspect_list):
    first_pos = sent_list.index(aspect_list[1]) #first position bc [0] is always the CLS token in transformers
    final_pos = []
    for i in range(0,len(aspect_list)-2):
        final_pos.append(first_pos+i)
    return final_pos

In [ ]:
#This class will construct a Torch dataset from the tagged sentences. Each instance in the dataset will contain the words, their position in the sentence and tags for those words.

from torch.utils import data
class ABSADataset(data.Dataset):
    def __init__(self, tagged_sents):
        sents, aspects, tags = [], [], [] # list of lists
        bugged = 0
        for sent in tagged_sents:
            try:
                sent_tokens = tokenizer.encode(sent[0])
                aspect_tokens = tokenizer.encode(sent[1])
                pos_aspects = get_pos(sent_tokens, aspect_tokens)
                tag = sent[2]
                sents.append(sent_tokens)
                aspects.append(pos_aspects)
                tags.append(tag)
            except:
                bugged+=1
        print("Ignoring {} Buggy Annotations".format(bugged))
        self.sents, self.aspects, self.tags = sents, aspects, tags

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, aspects, tags = self.sents[idx], self.aspects[idx], tag2idx[self.tags[idx]] # words, tags: string list
        return words, aspects, tags

#### B.4 Creating our Model for extracting embeddings

Let's wrap our loaded model in a class and then get embeddings for each aspect from the transformer. We will then use these embeddings to classify the sentiment of the aspect.

In [ ]:
#This class defines a network where we pass full sentences to a transformer for the entire context but only store the embeddings for the aspect terms.
#This is done by using the position index of the aspect term in the sentence we stored using our previous functions.

import torch
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = device
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

    def forward(self, sent, aspects, y):
        '''
        x: (N, T). int64
        y: (N, T). int64
        '''
        sent = torch.LongTensor(sent).to(self.device)
        aspects = torch.LongTensor(aspects).to(self.device)
        y = torch.LongTensor(y).to(self.device)
        input_ids = sent.unsqueeze(0)  # Batch size 1

        with torch.no_grad():
            outputs = self.model(input_ids)
            last_hidden_states = outputs.last_hidden_state[0]  # Get last hidden state (embeddings)
            start = 0
            end = len(last_hidden_states)-1
            context_window = 5 # define context window we're interested in (5 before and 5 after)

            if aspects[0]-context_window>0:
                start = aspects[0]-context_window
            if aspects[-1]+context_window<len(last_hidden_states)-1:
                end = aspects[-1]+context_window

            all_aspects = []
            for i in range(start,end):
                all_aspects.append(i)

            #For each index in all_aspects, it assigns the corresponding BERT embedding from the last_hidden_states tensor to the corresponding row in the bert_embeds tensor.
            #Each row of bert_embeds now holds the BERT embedding for the corresponding aspect.

            bert_embeds = torch.zeros(len(all_aspects),768).to(self.device)
            for i, aspect in enumerate(all_aspects):
                bert_embeds[i] = last_hidden_states[aspect]

            #  calculates the mean of the embeddings along axis 0 (the rows).
            #  This is done to obtain a single aggregated BERT embedding that represents the information from the context window around the aspects.

            embedding = torch.mean(bert_embeds, axis=0).to(self.device)

        return embedding

In [ ]:
#This function will run our network from the previous cell on our entire dataset, therefore extracting and storing embeddings for each aspect term.

def extract(model, iterator):
    model.eval()

    Words, Aspects, Y, Y_hat = [], [], [], [],
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            words, aspects, y = batch

            _, _, y_hat, _ = model(words, aspects, y)  # y_hat: (N, T) = predicted labels
            print(y_hat)

            Words.extend(words)
            Aspects.extend(aspects)
            Y.extend(y.numpy().tolist())
            Y_hat.extend([y_hat.cpu().numpy().tolist()])

    ## calc metric
    print(classification_report(Y, Y_hat))

#### B.4 Initialising the Model & exctracting the embeddings

Now we can begin to use all the massive functions we have defined in the previous sections and initialize first, our dataset, and then our model to be used for extracting the embeddings


In [ ]:
from transformers import AdamW

dataset = ABSADataset(raw_data)
data_iterator = data.DataLoader(dataset=dataset,
                             batch_size=1,
                             shuffle=False,
                             num_workers=0,
                             pin_memory=False)

Ignoring 0 Buggy Annotations


Perfect! Now let's initalize our network for extracting the embeddings

In [ ]:
model = Net()
model.to(device)

Net(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

Great, now let's run this model on our entire data to extract embeddings for all the aspects. Again, remember that this can take quite a while, so take a break!

In [ ]:
embeddings = []
import tqdm
for i, batch in tqdm.tqdm(enumerate(data_iterator)):
        words, aspects, y = batch
        embedding = model(words, aspects, y)
        embedding = embedding.cpu().numpy()
        y = int(y.cpu().numpy()[0])
        embeddings.append([embedding, y])

1322it [03:42,  5.94it/s]


#### B.5 Setting up a Machine Learning Classifier

Now that our embeddings are extracted. We can train a simple ML classifier to detect sentiment for an embedding. Let's first construct our data in the X (embeddings) and Y (labels) format of sklearn and then split it into a train and test split.

In [ ]:
X = [x[0] for x in embeddings]
Y = [x[1] for x in embeddings]

Let's see what a X and Y look like

In [ ]:
print("___________ Sample Embedding ___________ ")
print(X[100])
print("___________ Sample Sentiment ___________ ")
print(Y[100])

___________ Sample Embedding ___________ 
[ 8.20019469e-02 -1.47756776e-02  1.80196762e-01 -2.00758934e-01
  1.67954147e-01  5.63581944e-01 -3.01245362e-01  6.27293885e-01
  5.68736494e-01  1.96950197e-01 -5.03707789e-02  2.73313731e-01
 -1.32441416e-01 -1.81252480e-01 -5.35731912e-02  6.91305771e-02
  3.07307214e-01 -2.91986674e-01 -2.40733370e-01  3.92571956e-01
 -3.12691987e-01  9.61914510e-02 -1.14218239e-02  9.69716115e-04
  2.52802908e-01 -3.00708041e-02 -1.75336644e-01 -9.14047565e-03
  9.66977105e-02 -2.63373200e-02 -7.78606683e-02 -4.13104564e-01
 -1.50799185e-01  2.37354815e-01  3.91122729e-01 -2.59514928e-01
  3.26071084e-01  4.80993152e-01  3.50112617e-01  2.79841349e-02
  1.29022524e-01  7.90598541e-02  1.14769243e-01 -8.28820318e-02
 -4.21301186e-01 -2.27922589e-01  4.93911877e-02  3.11286021e-02
 -3.88510883e-01 -9.67323482e-02  5.47236800e-02 -1.18108466e-01
  7.88913965e-01  3.49447690e-02 -9.44073200e-02  1.70284018e-01
 -7.92331919e-02 -1.20594176e-02  3.56431007e-01

As you can see, X is a very long (768 dimensional) embedding of the aspect, while Y is the ID of the sentiment label (in this case 3)

Let's split X and Y now.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, shuffle=True, random_state=42, stratify=Y)

#### B.6 Train a ML Classifier on the Embeddings

Finally let's train a simple Linear SVM on the Embeddings. You can of course try to experiment with the classifier you want to use. Depending on the size of the data and the amount of labels, there might be other better options. Some commonly used classifiers include Decision Trees, Multi-layer Perceptron, Random Forests, etc.

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel="linear", C=0.025)

classifier.fit(X_train, y_train)


SVC(C=0.025, kernel='linear')

Our  model is trained! Let's find it out how accurate it is on the Test set.

In [ ]:
from sklearn.metrics import classification_report

predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.14      0.25        14
           1       0.62      0.62      0.62        64
           2       0.35      0.24      0.28        38
           3       0.67      0.88      0.76       130
           4       0.00      0.00      0.00        19

    accuracy                           0.63       265
   macro avg       0.53      0.38      0.38       265
weighted avg       0.58      0.63      0.58       265



As you can see, our model has an accuracy of 63% with less than 1000 labeled aspects labelled with sentiment used for training!